This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [3]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [4]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "muffin-vs-chihuahua/train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "muffin-vs-chihuahua/test"    # e.g. './muffin-vs-chihuahua/test'

In [5]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [6]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [7]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

l2_reg = tf.keras.regularizers.l2(0.001)

model = models.Sequential([
    layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),

    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.3),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.3),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),

    layers.Dense(128, activation='relu', kernel_regularizer=l2_reg),
    layers.Dropout(0.5),

    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [8]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 45s 367ms/step - accuracy: 0.5655 - loss: 0.9583 - val_accuracy: 0.7460 - val_loss: 0.7233
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 40s 339ms/step - accuracy: 0.7368 - loss: 0.6906 - val_accuracy: 0.7958 - val_loss: 0.5897
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 43s 361ms/step - accuracy: 0.7611 - loss: 0.5952 - val_accuracy: 0.7714 - val_loss: 0.5915
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 41s 341ms/step - accuracy: 0.7986 - loss: 0.5334 - val_accuracy: 0.8169 - val_loss: 0.5032
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 35s 291ms/step - accuracy: 0.8134 - loss: 0.4922 - val_accuracy: 0.8561 - val_loss: 0.4388
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 34s 288ms/step - accuracy: 0.8324 - loss: 0.4585 - val_accuracy: 0.8677 - val_loss: 0.4100
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 34s 286ms/step - accuracy: 0.8324 - loss: 0.4542 - val_accuracy: 0.8392 - val_loss: 0.4876
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 34s 288ms/step - accuracy: 0.8432 - loss: 0

In [10]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 114ms/step - accuracy: 0.8919 - loss: 0.3569
Test Accuracy: 0.8918918967247009


In [11]:
# SAVE THE MODEL
model.save('exercise_6_trained_model_improved.h5')

In [12]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0, 0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [13]:
# Example usage:
predict_image("run_1/run_1.jpg")
predict_image("run_1/run_2.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


Prediction: Chihuahua (confidence: 0.84)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Prediction: Muffin (confidence: 0.14)


Part 2: Custom dataset

In [14]:
# Organize the custom dataset into `cat` and `dog` subfolders
import os, shutil

def organize_cat_dog(base_path):
    """Walks `base_path` (e.g. 'cat-vs-dog') and ensures that each split
    ('train' and 'test') contains `cat` and `dog` directories. Files with names
    containing 'cat' or 'dog' (case-insensitive) are moved to the matching folder.
    Others go to an `unknown` folder for manual inspection.
    """
    for split in ('train', 'test'):
        split_dir = os.path.join(base_path, split)
        if not os.path.exists(split_dir):
            print(f'Warning: {split_dir} not found; skipping')
            continue
        cat_dir = os.path.join(split_dir, 'cat')
        dog_dir = os.path.join(split_dir, 'dog')
        unknown_dir = os.path.join(split_dir, 'unknown')
        os.makedirs(cat_dir, exist_ok=True)
        os.makedirs(dog_dir, exist_ok=True)
        os.makedirs(unknown_dir, exist_ok=True)

        moved = 0
        for fname in os.listdir(split_dir):
            fpath = os.path.join(split_dir, fname)
            # skip directories (including the newly created class folders)
            if os.path.isdir(fpath):
                continue
            lname = fname.lower()
            try:
                if 'cat' in lname:
                    shutil.move(fpath, os.path.join(cat_dir, fname))
                    moved += 1
                elif 'dog' in lname:
                    shutil.move(fpath, os.path.join(dog_dir, fname))
                    moved += 1
                else:
                    # fallback heuristic: file starts with c/d or single-letter labels
                    base = lname.split('.')[0]
                    if base.startswith('c') and any(ch.isalpha() for ch in base):
                        shutil.move(fpath, os.path.join(cat_dir, fname))
                        moved += 1
                    elif base.startswith('d') and any(ch.isalpha() for ch in base):
                        shutil.move(fpath, os.path.join(dog_dir, fname))
                        moved += 1
                    else:
                        shutil.move(fpath, os.path.join(unknown_dir, fname))
            except Exception as e:
                print(f'Error moving {fpath}: {e}')

        print(f'Organized {split_dir}: moved {moved} files (cat/dog -> subfolders).')

# Run the organizer on the provided dataset root
dataset_root = 'cat-vs-dog'
organize_cat_dog(dataset_root)

# Configure the custom dataset (cat vs dog)
custom_train_dir = "cat-vs-dog/train"
custom_test_dir = "cat-vs-dog/test"

# After organization, create generators using the existing ImageDataGenerator instances
train_generator_custom = train_datagen.flow_from_directory(
    custom_train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator_custom = train_datagen.flow_from_directory(
    custom_train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_generator_custom = test_datagen.flow_from_directory(
    custom_test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Organized cat-vs-dog\train: moved 0 files (cat/dog -> subfolders).
Organized cat-vs-dog\test: moved 0 files (cat/dog -> subfolders).
Found 20000 images belonging to 3 classes.
Found 5000 images belonging to 3 classes.
Found 12500 images belonging to 3 classes.


In [ ]:
# Build the same CNN architecture for the custom dataset and train it
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True,
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
l2_reg = tf.keras.regularizers.l2(0.001)

model_custom = models.Sequential([
    layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),

    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.3),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.3),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),

    layers.Dense(128, activation='relu', kernel_regularizer=l2_reg),
    layers.Dropout(0.5),

    layers.Dense(1, activation='sigmoid')
])

model_custom.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
# TRAINING THE CNN
history_custom = model_custom.fit(
    train_generator_custom,
    epochs=10,
    validation_data=val_generator_custom
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 142s 227ms/step - accuracy: 0.6618 - loss: 0.6224 - val_accuracy: 0.6814 - val_loss: 0.6050
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 120s 192ms/step - accuracy: 0.6648 - loss: 0.6193 - val_accuracy: 0.6946 - val_loss: 0.5990
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 124s 198ms/step - accuracy: 0.6641 - loss: 0.6178 - val_accuracy: 0.6830 - val_loss: 0.6016
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 121s 194ms/step - accuracy: 0.6698 - loss: 0.6143 - val_accuracy: 0.6884 - val_loss: 0.5865
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 127s 204ms/step - accuracy: 0.6697 - loss: 0.6150 - val_accuracy: 0.6916 - val_loss: 0.6035
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 322s 515ms/step - accuracy: 0.6730 - loss: 0.6132 - val_accuracy: 0.6902 - val_loss: 0.5792
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 285s 456ms/step - accuracy: 0.6705 - loss: 0.6102 - val_accuracy: 0.6602 - val_loss: 0.6249
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 144s 230ms/step - accuracy: 0.6758 -

In [18]:
# Evaluate the custom model on the custom test set
test_loss, test_acc = model_custom.evaluate(test_generator_custom)
print(f"Custom dataset test accuracy: {test_acc:.4f}")

391/391 ━━━━━━━━━━━━━━━━━━━━ 117s 299ms/step - accuracy: 0.0000e+00 - loss: 0.4526
Custom dataset test accuracy: 0.0000


In [19]:
# Save the trained custom model (final copy)
model_custom.save('exercise_6_custom_villarin.h5')

In [20]:
# Simple inference for the custom model (cat vs dog)
from tensorflow.keras.preprocessing import image

def predict_image_custom(img_path, model_path='exercise_6_custom_villarin.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0, 0]

    idx_to_label = {v: k for k, v in train_generator_custom.class_indices.items()}
    predicted_class = 1 if pred >= 0.5 else 0
    label = idx_to_label[predicted_class]
    print(f"Prediction: {label} (confidence: {pred:.2f})")

In [21]:
# Example usage
predict_image_custom("run_1/run_3.jpg")
predict_image_custom("run_1/run_4.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


Prediction: cat (confidence: 0.09)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Prediction: dog (confidence: 0.57)
